# Problem Setup

In [1]:
INITIAL_X = 0.01

def f(x):
    return -x**2

def df(x):
    return -2*x

CONSTRAINT = 6

def c(x):
    return 1.1*x**4 - (3*x**2) + 1

def dc(x):
    return 4.4*x**3 - (6*x)

In [2]:
from plotly.subplots import make_subplots
import numpy as np

xs = np.linspace(-2, 2)

fig = make_subplots(rows=2, cols=2, subplot_titles=("Objective function f(x)", "Constraint function c(x)", "", "Derivative constraint function c'(x)"))

fig.add_scatter(
    x=xs, y=f(xs), name="f(x)", row=1, col=1
)

fig.add_scatter(
    x=xs, y=df(xs), name="f'(x)", row=1, col=1
)

fig.add_scatter(
    x=xs, y=c(xs), name="c(x)", row=1, col=2
)

fig.add_scatter(
    x=[xs[0], xs[-1]], 
    y=[CONSTRAINT, CONSTRAINT], 
    name="Constraint lower bound", 
    mode="lines",
    line={"dash": "dash"},
    row=1, 
    col=2,
)

fig.add_scatter(
    x=xs, y=dc(xs), name="c'(x)", row=2, col=2
)

fig.add_vline(x=INITIAL_X, line_dash="dash")

fig.show()


In [3]:
print(f"f'(x0) = {df(INITIAL_X)}")
print(f"c'(x0) = {dc(INITIAL_X)}")

f'(x0) = -0.02
c'(x0) = -0.059995599999999996


## QPP Plots

The Quadratic Programming Problem is outlined in Equation 4. The QPP will have no solution if the 'QPP constraint | x0' line is never greater than 'QPP constraint lower bound'.

In [9]:
import plotly.graph_objects as go

def Q_for_x0(delta):
    # this is reduced because we have a 1D function and no hessian because 1D identity is 1
    return f(INITIAL_X) + delta*df(INITIAL_X) + 0.5*delta**2

def constraint_for_x0(delta):
    # remember constraint is c(x) - lower_bound
    return dc(INITIAL_X)*delta + (c(INITIAL_X) - CONSTRAINT)

deltas = np.linspace(-2, 2)

fig = go.Figure()

fig.add_scatter(x=deltas, y=Q_for_x0(deltas), name="Q(delta) | x0")
fig.add_scatter(x=deltas, y=constraint_for_x0(deltas), name="QPP constraint | x0")
fig.add_scatter(
    x=[deltas[0], deltas[-1]], 
    y=[0, 0], 
    name="QPP constraint lower bound",
    mode="lines",
    line={"dash": "dash"}
)

In [10]:
print(f"QPP will be solvable for delta >= {(CONSTRAINT - c(INITIAL_X))/dc(INITIAL_X)}")

QPP will be solvable for delta >= -83.34444507597225


So long as the orange line has a gradient, a solution to the QPP should be possible so it is unclear why CVXPY cannot solve this problem. More investigation is needed. 

## VMCON Solve

In [11]:
from pyvmcon import Problem

problem = Problem(
    f=f,
    df=df,
    equality_constraints=[],
    # inequality_constraints=[lambda x: CONSTRAINT - c(x)],
    inequality_constraints=[lambda x: c(x) - CONSTRAINT],
    # inequality_constraints=[],
    dequality_constraints=[],
    # dinequality_constraints=[lambda x: -dc(x)]
    dinequality_constraints=[lambda x: dc(x)]
    # dinequality_constraints=[]
)

In [12]:
from pyvmcon import solve

x, _, _, result = solve(problem, np.array([INITIAL_X]))

SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

In [13]:
x

NameError: name 'x' is not defined